# 03 - modelo con preprocesado imputación Estrato 2 y SVM

## Objetivo del notebook

En este notebook implementamos una variante de modelo predictivo usando:

- El preprocesamiento basado en la imputación de valores faltantes de la variable FAMI_ESTRATOVIVIENDA asignando la categoría "Estrato 2".

- La codificación one-hot de dicha variable.

- Un clasificador SVM (Support Vector Machine) para realizar la predicción sobre RENDIMIENTO_GLOBAL.


## Carga de librerías y datos

In [25]:
# CARGA DE DATOS Y LIBRERÍAS

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.svm import SVR


In [26]:
# Cargar los datos originales
df = pd.read_csv("train.csv")
print("Shape original:", df.shape)
display(df.head())

Shape original: (692500, 12)


,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo


## Muestra de trabajo

Dado que el dataset completo es muy grande, para fines de pruebas preliminares trabajamos con una muestra aleatoria de 2000 registros, tal como lo hicimos en los notebooks anteriores.

In [27]:
# REDUCCIÓN DE MUESTRA (manteniendo la misma muestra de los notebooks anteriores)
df = df.sample(n=2000, random_state=42).reset_index(drop=True)
print("Shape muestra:", df.shape)

Shape muestra: (2000, 12)


In [28]:

# IMPUTACIÓN ESTRATO 2 EN 'FAMI_ESTRATOVIVIENDA'

col = "FAMI_ESTRATOVIVIENDA"
df[col] = df[col].fillna("Estrato 2")

In [29]:
# TRATAMIENTO DE TODAS LAS VARIABLES CATEGÓRICAS

# Listado de columnas categóricas a transformar
cat_cols = [
    "ESTU_VALORMATRICULAUNIVERSIDAD",
    "ESTU_HORASSEMANATRABAJA",
    "FAMI_ESTRATOVIVIENDA",
    "FAMI_TIENEINTERNET",
    "FAMI_EDUCACIONPADRE",
    "FAMI_EDUCACIONMADRE",
    "ESTU_PAGOMATRICULAPROPIO"
]

# Reemplazamos los NaN por la categoría más frecuente de cada columna
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

# Hacemos el one-hot encoding de forma automática
df_encoded = pd.get_dummies(df[cat_cols])

In [30]:
# UNIMOS VARIABLES NUMÉRICAS + ONE HOT
# (PERIODO e ID también pueden ser considerados si deseas)
num_cols = ["PERIODO"]

X = pd.concat([df[num_cols], df_encoded], axis=1)
print("Shape final de X:", X.shape)

Shape final de X: (2000, 49)


In [31]:
# TRANSFORMACIÓN DE TARGET
rendimiento_map = {
    'bajo': 0,
    'medio-bajo': 1,
    'medio-alto': 2,
    'alto': 3
}
y = df["RENDIMIENTO_GLOBAL"].map(rendimiento_map)

In [32]:
# ENTRENAMIENTO Y VALIDACIÓN CON SVM

# Definimos el modelo
estimator = SVR()

# Definimos la validación cruzada
cv = ShuffleSplit(n_splits=30, test_size=0.3, random_state=42)

# Métrica de evaluación
scoring = make_scorer(mean_absolute_error)

# Ejecutamos cross validation
scores = cross_val_score(estimator, X, y, cv=cv, scoring=scoring)

# Mostramos resultados
print("MAE promedio: %.3f (±%.3f)" % (scores.mean(), scores.std()))

MAE promedio: 1.014 (±0.022)
